In [4]:
%matplotlib inline
import numpy as np
np.random.seed(555)
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import time

from statsmodels.graphics import tsaplots
from statsmodels.tsa import stattools
from statsmodels.tsa import ar_model

import statsmodels.api as sm # version 0.8.0以上
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [2]:
status = pd.read_csv('../../../status.csv')
#statusのyear, month, dayを結合してdatetime型に
status['date'] = status['year'].astype(str) + '/' + status['month'].astype(str).str.zfill(2).astype(str) + '/' + status['day'].astype(str).str.zfill(2).astype(str) + '/' + status['hour'].astype(str).str.zfill(2).astype(str)
status['date'] = pd.to_datetime(status['date'], format='%Y/%m/%d/%H', infer_datetime_format=True)

#曜日を追加するための関数を定義
def get_weekday_jp(dt):
    w_list = ['月曜日', '火曜日', '水曜日', '木曜日', '金曜日', '土曜日', '日曜日']
    return(w_list[dt.weekday()])

#dateから曜日情報を取得
status["weekday"] = status["date"].apply(get_weekday_jp)

main_df = status[['date','hour', 'station_id', 'bikes_available', 'weekday', 'day']]

#カテゴリ変数をダミー変数化
main_df = pd.get_dummies(main_df)

stationid_set = [0, 15, 32, 36, 38, 40, 43, 49, 54, 56] 
dt_now = datetime.datetime.now()

print(dt_now)

2022-01-20 23:08:01.716859


In [3]:
for station_id in stationid_set:
    train_X = main_df[main_df['station_id'] == station_id]

    train_learn = train_X[train_X['date'] < '2013-10-01']
    train_learn = train_learn[train_learn['date'] >= '2013-09-01']
    train_learn_x = train_learn.drop("station_id",axis=1)
    train_learn_x = train_learn_x.set_index('date')
    train_exog =  train_learn_x.drop("bikes_available",axis=1)
    test = main_df[main_df['station_id'] == station_id]
    test = test[test['date'] <= '2013-10-08 00:00:00']
    test = test[test['date'] >= '2013-10-01']
    test = test.set_index('date')
    test_exog = test.drop("station_id",axis=1)
    test_exog =  test_exog.drop("bikes_available",axis=1)

    p, d, q, sp, sd, sq = 24, 1, 24, 1, 1, 1
    seasonal = 48

    # SARIMAモデルの当てはめいろいろ
    result_sarima = SARIMAX(
        endog = train_learn_x["bikes_available"], 
        exog = train_exog,
        order=(p,d,q), 
        seasonal_order=(sp,sd,sq,seasonal), 
        enforce_stationarity = False, 
        enforce_invertibility = False
    ).fit()
    # 結果確認
    print(result_sarima.summary())    
    train_file_name = "Sarimax_24_predict/Sarimax_train_stationid" + str(station_id) + "_201309.csv" 
    result_sarima.fittedvalues.to_csv(train_file_name)
    predict_sarima = result_sarima.predict('2013-10-01', '2013-10-08', exog=test_exog, dynamic=True)
    test_file_name = "Sarimax_24_predict/Sarimax_test_stationid" + str(station_id) + "_201309.csv" 
    predict_sarima.to_csv(test_file_name)
    
    print(dt_now)

C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                                       SARIMAX Results                                        
Dep. Variable:                        bikes_available   No. Observations:                  720
Model:             SARIMAX(24, 1, 24)x(1, 1, [1], 48)   Log Likelihood               -1090.204
Date:                                Tue, 11 Jan 2022   AIC                           2300.409
Time:                                        22:36:48   BIC                           2564.024
Sample:                                    09-01-2013   HQIC                          2403.044
                                         - 09-30-2013                                         
Covariance Type:                                  opg                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
hour        -6.885e-05   3579.419  -1.92e-08      1.000   -7015.533    7015.532


C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                                       SARIMAX Results                                        
Dep. Variable:                        bikes_available   No. Observations:                  720
Model:             SARIMAX(24, 1, 24)x(1, 1, [1], 48)   Log Likelihood                -724.411
Date:                                Tue, 11 Jan 2022   AIC                           1568.821
Time:                                        22:44:20   BIC                           1832.437
Sample:                                    09-01-2013   HQIC                          1671.457
                                         - 09-30-2013                                         
Covariance Type:                                  opg                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
hour         1.868e-05   4836.092   3.86e-09      1.000   -9478.567    9478.567


C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


...

                                       SARIMAX Results                                        
Dep. Variable:                        bikes_available   No. Observations:                  720
Model:             SARIMAX(24, 1, 24)x(1, 1, [1], 48)   Log Likelihood               -1317.714
Date:                                Tue, 11 Jan 2022   AIC                           2755.428
Time:                                        23:26:40   BIC                           3019.043
Sample:                                    09-01-2013   HQIC                          2858.063
                                         - 09-30-2013                                         
Covariance Type:                                  opg                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
hour         2.052e-05    611.263   3.36e-08      1.000   -1198.053    1198.053


C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                                       SARIMAX Results                                        
Dep. Variable:                        bikes_available   No. Observations:                  720
Model:             SARIMAX(24, 1, 24)x(1, 1, [1], 48)   Log Likelihood               -1175.008
Date:                                Tue, 11 Jan 2022   AIC                           2470.015
Time:                                        23:32:47   BIC                           2733.631
Sample:                                    09-01-2013   HQIC                          2572.651
                                         - 09-30-2013                                         
Covariance Type:                                  opg                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
hour           -0.0005   2534.953  -2.12e-07      1.000   -4968.417    4968.416


C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                                       SARIMAX Results                                        
Dep. Variable:                        bikes_available   No. Observations:                  720
Model:             SARIMAX(24, 1, 24)x(1, 1, [1], 48)   Log Likelihood               -1045.704
Date:                                Tue, 11 Jan 2022   AIC                           2211.408
Time:                                        23:39:13   BIC                           2475.023
Sample:                                    09-01-2013   HQIC                          2314.044
                                         - 09-30-2013                                         
Covariance Type:                                  opg                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
hour           -0.0001   2892.439  -4.06e-08      1.000   -5669.077    5669.077


In [4]:
for station_id in stationid_set:
    train_X = main_df[main_df['station_id'] == station_id]

    train_learn = train_X[train_X['date'] < '2014-02-01']
    train_learn = train_learn[train_learn['date'] >= '2014-01-01']
    train_learn_x = train_learn.drop("station_id",axis=1)
    train_learn_x = train_learn_x.set_index('date')
    train_exog =  train_learn_x.drop("bikes_available",axis=1)
    test = main_df[main_df['station_id'] == station_id]
    test = test[test['date'] <= '2014-02-08 00:00:00']
    test = test[test['date'] >= '2014-02-01']
    test = test.set_index('date')
    test_exog = test.drop("station_id",axis=1)
    test_exog =  test_exog.drop("bikes_available",axis=1)

    p, d, q, sp, sd, sq = 24, 1, 24, 1, 1, 1
    seasonal = 48

    # SARIMAモデルの当てはめいろいろ
    result_sarima = SARIMAX(
        endog = train_learn_x["bikes_available"], 
        exog = train_exog,
        order=(p,d,q), 
        seasonal_order=(sp,sd,sq,seasonal), 
        enforce_stationarity = False, 
        enforce_invertibility = False
    ).fit()
    # 結果確認
    print(result_sarima.summary())    
    train_file_name = "Sarimax_24_predict/Sarimax_train_stationid" + str(station_id) + "_201401.csv" 
    result_sarima.fittedvalues.to_csv(train_file_name)
    predict_sarima = result_sarima.predict('2014-02-01', '2014-02-08', exog=test_exog, dynamic=True)
    test_file_name = "Sarimax_24_predict/Sarimax_test_stationid" + str(station_id) + "_201401.csv" 
    predict_sarima.to_csv(test_file_name)
    dt_now = datetime.datetime.now()
    print(dt_now)

C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                                       SARIMAX Results                                        
Dep. Variable:                        bikes_available   No. Observations:                  744
Model:             SARIMAX(24, 1, 24)x(1, 1, [1], 48)   Log Likelihood               -1108.626
Date:                                Tue, 11 Jan 2022   AIC                           2337.252
Time:                                        23:46:18   BIC                           2603.228
Sample:                                    01-01-2014   HQIC                          2440.624
                                         - 01-31-2014                                         
Covariance Type:                                  opg                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
hour           -0.0006   3450.750  -1.88e-07      1.000   -6763.346    6763.345


C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                                       SARIMAX Results                                        
Dep. Variable:                        bikes_available   No. Observations:                  744
Model:             SARIMAX(24, 1, 24)x(1, 1, [1], 48)   Log Likelihood                -469.399
Date:                                Tue, 11 Jan 2022   AIC                           1058.799
Time:                                        23:53:01   BIC                           1324.775
Sample:                                    01-01-2014   HQIC                          1162.171
                                         - 01-31-2014                                         
Covariance Type:                                  opg                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
hour            0.0001   2494.786   4.45e-08      1.000   -4889.690    4889.690


C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


...

                                       SARIMAX Results                                        
Dep. Variable:                        bikes_available   No. Observations:                  744
Model:             SARIMAX(24, 1, 24)x(1, 1, [1], 48)   Log Likelihood               -1357.587
Date:                                Wed, 12 Jan 2022   AIC                           2835.174
Time:                                        00:35:39   BIC                           3101.150
Sample:                                    01-01-2014   HQIC                          2938.546
                                         - 01-31-2014                                         
Covariance Type:                                  opg                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
hour           -0.0004   3268.439  -1.21e-07      1.000   -6406.023    6406.022


C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                                       SARIMAX Results                                        
Dep. Variable:                        bikes_available   No. Observations:                  744
Model:             SARIMAX(24, 1, 24)x(1, 1, [1], 48)   Log Likelihood               -1320.883
Date:                                Wed, 12 Jan 2022   AIC                           2761.766
Time:                                        00:43:59   BIC                           3027.742
Sample:                                    01-01-2014   HQIC                          2865.137
                                         - 01-31-2014                                         
Covariance Type:                                  opg                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
hour           -0.0002   3851.301  -4.69e-08      1.000   -7548.412    7548.412


C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                                       SARIMAX Results                                        
Dep. Variable:                        bikes_available   No. Observations:                  744
Model:             SARIMAX(24, 1, 24)x(1, 1, [1], 48)   Log Likelihood               -1190.670
Date:                                Wed, 12 Jan 2022   AIC                           2501.339
Time:                                        00:49:47   BIC                           2767.316
Sample:                                    01-01-2014   HQIC                          2604.711
                                         - 01-31-2014                                         
Covariance Type:                                  opg                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
hour            0.0007   5246.524   1.35e-07      1.000   -1.03e+04    1.03e+04


In [5]:
for station_id in stationid_set:
    train_X = main_df[main_df['station_id'] == station_id]
    train_learn = train_X[train_X['date'] < '2014-06-01']
    train_learn = train_learn[train_learn['date'] >= '2014-05-01']
    train_learn_x = train_learn.drop("station_id",axis=1)
    train_learn_x = train_learn_x.set_index('date')
    train_exog =  train_learn_x.drop("bikes_available",axis=1)
    test = main_df[main_df['station_id'] == station_id]
    test = test[test['date'] <= '2014-06-08 00:00:00']
    test = test[test['date'] >= '2014-06-01']
    test = test.set_index('date')
    test_exog = test.drop("station_id",axis=1)
    test_exog =  test_exog.drop("bikes_available",axis=1)

    p, d, q, sp, sd, sq = 24, 1, 24, 1, 1, 1
    seasonal = 48

    # SARIMAモデルの当てはめいろいろ
    result_sarima = SARIMAX(
        endog = train_learn_x["bikes_available"], 
        exog = train_exog,
        order=(p,d,q), 
        seasonal_order=(sp,sd,sq,seasonal), 
        enforce_stationarity = False, 
        enforce_invertibility = False
    ).fit()
    # 結果確認
    print(result_sarima.summary())    
    train_file_name = "Sarimax_24_predict/Sarimax_train_stationid" + str(station_id) + "_201405.csv" 
    result_sarima.fittedvalues.to_csv(train_file_name)
    predict_sarima = result_sarima.predict('2014-06-01', '2014-06-08', exog=test_exog, dynamic=True)
    test_file_name = "Sarimax_24_predict/Sarimax_test_stationid" + str(station_id) + "_201405.csv" 
    predict_sarima.to_csv(test_file_name)
    dt_now = datetime.datetime.now()
    print(dt_now)

C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                                       SARIMAX Results                                        
Dep. Variable:                        bikes_available   No. Observations:                  744
Model:             SARIMAX(24, 1, 24)x(1, 1, [1], 48)   Log Likelihood               -1248.762
Date:                                Wed, 12 Jan 2022   AIC                           2617.524
Time:                                        00:57:19   BIC                           2883.500
Sample:                                    05-01-2014   HQIC                          2720.896
                                         - 05-31-2014                                         
Covariance Type:                                  opg                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
hour            0.0003    528.146   4.94e-07      1.000   -1035.146    1035.147


C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                                       SARIMAX Results                                        
Dep. Variable:                        bikes_available   No. Observations:                  744
Model:             SARIMAX(24, 1, 24)x(1, 1, [1], 48)   Log Likelihood                -431.403
Date:                                Wed, 12 Jan 2022   AIC                            982.807
Time:                                        01:04:57   BIC                           1248.783
Sample:                                    05-01-2014   HQIC                          1086.179
                                         - 05-31-2014                                         
Covariance Type:                                  opg                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
hour         1.407e-05   1.24e+04   1.14e-09      1.000   -2.43e+04    2.43e+04


C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


...

                                       SARIMAX Results                                        
Dep. Variable:                        bikes_available   No. Observations:                  744
Model:             SARIMAX(24, 1, 24)x(1, 1, [1], 48)   Log Likelihood               -1558.227
Date:                                Wed, 12 Jan 2022   AIC                           3236.454
Time:                                        01:47:57   BIC                           3502.431
Sample:                                    05-01-2014   HQIC                          3339.826
                                         - 05-31-2014                                         
Covariance Type:                                  opg                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
hour           -0.0005   4178.445  -1.22e-07      1.000   -8189.602    8189.601


C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                                       SARIMAX Results                                        
Dep. Variable:                        bikes_available   No. Observations:                  744
Model:             SARIMAX(24, 1, 24)x(1, 1, [1], 48)   Log Likelihood               -1275.063
Date:                                Wed, 12 Jan 2022   AIC                           2670.126
Time:                                        01:54:15   BIC                           2936.103
Sample:                                    05-01-2014   HQIC                          2773.498
                                         - 05-31-2014                                         
Covariance Type:                                  opg                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
hour          8.02e-05   1104.215   7.26e-08      1.000   -2164.221    2164.221


C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                                       SARIMAX Results                                        
Dep. Variable:                        bikes_available   No. Observations:                  744
Model:             SARIMAX(24, 1, 24)x(1, 1, [1], 48)   Log Likelihood               -1237.916
Date:                                Wed, 12 Jan 2022   AIC                           2595.832
Time:                                        02:01:03   BIC                           2861.809
Sample:                                    05-01-2014   HQIC                          2699.204
                                         - 05-31-2014                                         
Covariance Type:                                  opg                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
hour         2.332e-06   4142.210   5.63e-10      1.000   -8118.583    8118.583


In [5]:
stationid_set = [0]
start = time.time()
for station_id in stationid_set:
    train_X = main_df[main_df['station_id'] == station_id]

    train_learn = train_X[train_X['date'] < '2013-10-01']
    train_learn = train_learn[train_learn['date'] >= '2013-09-01']
    train_learn_x = train_learn.drop("station_id",axis=1)
    train_learn_x = train_learn_x.set_index('date')
    train_exog =  train_learn_x.drop("bikes_available",axis=1)
    test = main_df[main_df['station_id'] == station_id]
    test = test[test['date'] <= '2013-10-08 00:00:00']
    test = test[test['date'] >= '2013-10-01']
    test = test.set_index('date')
    test_exog = test.drop("station_id",axis=1)
    test_exog =  test_exog.drop("bikes_available",axis=1)

    p, d, q, sp, sd, sq = 24, 1, 24, 1, 1, 1
    seasonal = 48

    # SARIMAモデルの当てはめいろいろ
    result_sarima = SARIMAX(
        endog = train_learn_x["bikes_available"], 
        exog = train_exog,
        order=(p,d,q), 
        seasonal_order=(sp,sd,sq,seasonal), 
        enforce_stationarity = False, 
        enforce_invertibility = False
    ).fit()
    # 結果確認
    print(result_sarima.summary())    
    train_file_name = "Sarimax_24_predict/Sarimax_train_stationid" + str(station_id) + "_201309.csv" 
    result_sarima.fittedvalues.to_csv(train_file_name)
    predict_sarima = result_sarima.predict('2013-10-01', '2013-10-08', exog=test_exog, dynamic=True)
    test_file_name = "Sarimax_24_predict/Sarimax_test_stationid" + str(station_id) + "_201309.csv" 
    predict_sarima.to_csv(test_file_name)
    
elapsed_time = time.time() - start
print(elapsed_time)

C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
C:\Users\pytho\anaconda3\envs\py-test\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                                       SARIMAX Results                                        
Dep. Variable:                        bikes_available   No. Observations:                  720
Model:             SARIMAX(24, 1, 24)x(1, 1, [1], 48)   Log Likelihood               -1090.204
Date:                                Thu, 20 Jan 2022   AIC                           2300.409
Time:                                        23:18:21   BIC                           2564.024
Sample:                                    09-01-2013   HQIC                          2403.044
                                         - 09-30-2013                                         
Covariance Type:                                  opg                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
hour        -6.885e-05   3579.419  -1.92e-08      1.000   -7015.533    7015.532
